In [19]:
import discogs_client
import pandas as pd
import time
from utils import lower_and_remove_diacritics, similar

In [20]:
d = discogs_client.Client('datamilas/0.1', user_token='BDGKHOLMZJxywoowKXvyRhXeqJhkezYxwVjxvBJR')

Already scraped
artist_name = "Severina"
artist_id = 473824

artist_name = "Mia Dimšić"
artist_id = 5416029

artist_name = "Jelena Rozga"
artist_id = 1546627

artist_name = "Petar Grašo"
artist_id = 793012


artist_name = "Đani Maršan"
artist_id = 929048

In [21]:
artist_name = "Alen Vitasović"
artist_id = 345900

In [22]:
artist_object = d.artist(artist_id)
artist_discogs = artist_object.name
artist_object

<Artist 345900 'Alen Vitasović'>

In [23]:
albums = artist_object.releases


In [24]:
df_discogs = pd.DataFrame(columns=["Artist"])

In [25]:
i=0
for album in albums:
    if album.data["role"] == "Main":
        print(album)
        time.sleep(0.5)
        for track in album.tracklist:
            if len(track.artists)==0 or (artist_object in track.artists):
                df_discogs.loc[i, "Artist"] = artist_name
                df_discogs.loc[i, "Artist_discogs"] = artist_discogs
                df_discogs.loc[i, "Song_discogs"] = track.title
                if "duration" in track.data.keys(): df_discogs.loc[i, "Song_duration_discogs"] = track.data["duration"]
                if "extraartists" in track.data.keys():
                    for ea in track.data["extraartists"]:
                        roles = ea["role"].split(", ")

                        for role in roles:
                            try: df_discogs.loc[i, f"{role}_discogs"] = df_discogs.loc[i, role]+"/"+ea["name"]
                            except: df_discogs.loc[i, f"{role}_discogs"] = ea["name"]
                df_discogs.loc[i, "Album_title_discogs"] = album.title
                df_discogs.loc[i, "Album_year_discogs"] = album.year
                df_discogs.loc[i, "Album_genres_discogs"] = str(album.genres)

                i+=1

<Release 4803058 'Ne Moren Bez Nje'>
<Master 393539 'Gušti Su Gušti'>
<Master 1442330 'Svi Festivali Plus Koncert'>
<Master 1342684 'Come Va?'>
<Release 7647856 'Ja Ne Gren'>
<Release 12699050 'Grih'>
<Release 5616168 'Zlatna Kolekcija'>
<Release 5816047 'Put Ka Sreći • Hrvatski Glazbenici Za Ivu Josipovića'>
<Release 7478973 'Sve Spod Mi Raklja'>
<Release 12298497 'The Best Of Collection'>
<Release 17178472 'Oštarije Su Mi Zaprli'>


In [26]:
df_discogs["Album_year_discogs"] = df_discogs["Album_year_discogs"].astype(int)
df_discogs["Album_year_title_discogs"] = df_discogs["Album_year_discogs"].astype(str)+"-"+df_discogs["Album_title_discogs"]
df_discogs["All_albums_discogs"] = df_discogs.groupby("Song_discogs")["Album_year_title_discogs"].transform(lambda x: '/'.join(x))

In [29]:
df_discogs.sort_values("Album_year_discogs").reset_index(drop=True, inplace=True)
df_discogs = df_discogs.drop_duplicates(subset=["Song_discogs"])
df_discogs = df_discogs.reset_index(drop=True)

In [30]:
columns_to_take = [col for col in df_discogs.columns if col in ['Artist_discogs', 'Song_discogs', 'Song_duration_discogs',
       'Album_title_discogs', 'Album_year_discogs', 'Album_genres_discogs','All_albums_discogs',
       'Lyrics By_discogs', 'Music By_discogs', 'Featuring_discogs']]


In [32]:
df = pd.read_csv(f"data/{artist_name}_tekstovinet_lyricstranslate_diskografija_cuspajz.csv")

In [33]:
similarity_threshold = 0.9
song_names = [lower_and_remove_diacritics(s).split('(')[0] for s in df_discogs.Song_discogs]
for row in df.iterrows():
    song_title = lower_and_remove_diacritics(row[1].Song_tekstovinet).split('(')[0]
    similarities = [similar(song_title, s) for s in song_names]
    max_similarity = max(similarities)
    
    if max_similarity>similarity_threshold:
        df.loc[row[0], columns_to_take] = df_discogs.loc[similarities.index(max_similarity)][columns_to_take]



In [34]:
df[["Album_1_diskografija", "Album_year_discogs", "Album_title_discogs", "All_albums_discogs"]].dropna()

,Album_1_diskografija,Album_year_discogs,Album_title_discogs,All_albums_discogs
1,1995 - Svi festivali,1995.0,Svi Festivali Plus Koncert,1995-Svi Festivali Plus Koncert/2010-Zlatna Ko...
3,1997 - Come va?,1997.0,Come Va?,1997-Come Va?
7,1994 - Gušti su gušti,1994.0,Gušti Su Gušti,1994-Gušti Su Gušti/1995-Svi Festivali Plus Ko...
8,2000 - Dora 2000,2000.0,Ja Ne Gren,2000-Ja Ne Gren/2010-Zlatna Kolekcija
10,1994 - Gušti su gušti,1995.0,Svi Festivali Plus Koncert,1995-Svi Festivali Plus Koncert/2010-Zlatna Ko...
12,1995 - Svi festivali,1995.0,Svi Festivali Plus Koncert,1995-Svi Festivali Plus Koncert
19,1995 - Svi festivali,1995.0,Svi Festivali Plus Koncert,1995-Svi Festivali Plus Koncert/2000-Ja Ne Gre...
21,1995 - Svi festivali,1995.0,Svi Festivali Plus Koncert,1995-Svi Festivali Plus Koncert/2010-Zlatna Ko...
22,1994 - Gušti su gušti,1993.0,Ne Moren Bez Nje,1993-Ne Moren Bez Nje/1994-Gušti Su Gušti/1995...
23,2006 - Dora 2006,2010.0,Zlatna Kolekcija,2010-Zlatna Kolekcija


In [35]:
df[["Album_lyricstranslate", "Album_year_discogs", "Album_title_discogs"]].dropna()

,Album_lyricstranslate,Album_year_discogs,Album_title_discogs
18,CD Single,2003.0,Grih


In [36]:
df.to_csv(f"data/{artist_name}_final.csv", index=False)

In [37]:
df_discogs.to_csv(f"data/{artist_name}_discogs.csv", index=False)